#### Set up Dataset - A binary subset of CIFAR 10

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications import DenseNet121, VGG16, InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Create a binary subset for classes 'cat' (label 3) and 'dog' (label 5)
binary_classes = [3, 5]
train_filter = np.isin(y_train, binary_classes).reshape(-1)
test_filter = np.isin(y_test, binary_classes).reshape(-1)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Map labels to binary (0 and 1)
y_train = (y_train == 5).astype(int)
y_test = (y_test == 5).astype(int)

# Resize images to match input size for pretrained models (224x224)
IMG_SIZE = (224, 224)

# Resizing function for CIFAR-10 images
def preprocess_images(images):
    return np.array([tf.image.resize(image, IMG_SIZE).numpy() for image in images])

x_train_resized = preprocess_images(x_train)
x_test_resized = preprocess_images(x_test)


In [2]:
x_train_resized.shape

(10000, 224, 224, 3)

#### Preprocessing and Augmentation

In [3]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow(x_train_resized, y_train, batch_size=32)
test_gen = test_datagen.flow(x_test_resized, y_test, batch_size=32)

#### Creating Models
* DenseNet121
* VGG16
* InceptionResNetV2

In [4]:
def create_model(base_model_class, input_shape=(224, 224, 3)):
    base_model = base_model_class(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model layers

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Initialize models
densenet_model = create_model(DenseNet121)
vgg_model = create_model(VGG16)
inception_resnet_model = create_model(InceptionResNetV2)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [6]:
# Training configuration
epochs = 50
learning_rate = 1e-4
patience = 5

# Function to compile, train, and return training history
def compile_and_train(model, train_gen, val_gen, epochs, learning_rate, model_name):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    checkpoint = ModelCheckpoint(f'{model_name}_best_model.keras', monitor='val_accuracy', save_best_only=True)

    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=[early_stopping, checkpoint])
    return history

# Train each model and collect histories
densenet_history = compile_and_train(densenet_model, train_gen, test_gen, epochs, learning_rate, 'DenseNet121')
vgg_history = compile_and_train(vgg_model, train_gen, test_gen, epochs, learning_rate, 'VGG16')
inception_resnet_history = compile_and_train(inception_resnet_model, train_gen, test_gen, epochs, learning_rate, 'InceptionResNetV2')


/Users/thory/miniconda3/envs/tiny-ml/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.6199 - loss: 0.6530

/Users/thory/miniconda3/envs/tiny-ml/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 283s 897ms/step - accuracy: 0.6200 - loss: 0.6529 - val_accuracy: 0.7355 - val_loss: 0.5385
Epoch 2/50
 62/313 ━━━━━━━━━━━━━━━━━━━━ 3:15 781ms/step - accuracy: 0.7389 - loss: 0.5507

KeyboardInterrupt: 